# Spam/Jailbreak Classification

---

## Dependencies

### Modules

In [ ]:
%pip install fastai

In [ ]:
%pip install torch

In [ ]:
%pip install transformers

In [ ]:
%pip install datasets

In [ ]:
%pip install tokenizers

In [ ]:
%pip install scikit-learn

In [ ]:
%pip install matplotlib

In [ ]:
%pip install spacy

In [ ]:
%pip install evaluate

In [ ]:
%pip install accelerate

### Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from fastai.text.all import *
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import numpy as np
import evaluate
import os

2025-08-07 09:46:53.760338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754560013.953079      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754560014.009483      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


---

## Data

190K+ Spam | Ham Email Dataset for Classification: https://www.kaggle.com/datasets/meruvulikith/190k-spam-ham-email-dataset-for-classification

Emails for spam or ham classification (Trec 2007): https://www.kaggle.com/datasets/bayes2003/emails-for-spam-or-ham-classification-trec-2007?select=email_text.csv

### Filtering

In [3]:
directory = "data"
try:
    os.mkdir(directory)
    print(f"Directory '{directory}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory}' already exists.")

Directory 'data' created successfully.


Download 2 datasets into "data" folder

In [7]:
# df_a = pd.read_csv("data/email_text.csv")
# df_b = pd.read_csv("data/spam_Emails_data.csv")

#kaggle 
df_a = pd.read_csv("/kaggle/input/emails-for-spam-or-ham-classification-trec-2007/email_text.csv")
df_b = pd.read_csv("/kaggle/input/190k-spam-ham-email-dataset-for-classification/spam_Emails_data.csv")

In [8]:
df_b['label'] = df_b['label'].map({"Spam": 1, "Ham": 0})

In [9]:
df_b['label'].unique()

array([1, 0])

In [10]:
merged = pd.concat([df_a[['label', 'text']], df_b[['label', 'text']]], ignore_index=True)
merged

,label,text
0,1,do you feel the pressure to perform and not rising to the occasion try v ia gr a your anxiety will be a thing of the past and you will be back to your old self
1,0,hi i've just updated from the gulus and i check on other mirrors it seems there is a little typo in debian readme file example http gulus usherbrooke ca debian readme ftp ftp fr debian org debian readme testing or lenny access this release through dists testing the current tested development snapshot is named etch packages which have been tested in unstable and passed automated tests propogate to this release etch should be replace by lenny like in the readme html yan morin consultant en logiciel libre yan morin savoirfairelinux com escapenumber escapenumber escapenumber to unsubscribe ema...
2,1,mega authenticv i a g r a discount pricec i a l i s discount pricedo not miss it click here http www moujsjkhchum com
3,1,hey billy it was really fun going out the other night and talking while we were out you said that you felt insecure about your manhood i noticed in the toilets you were quite small in that area but not to worry that website that i was telling you about is my secret weapon to an extra escapenumber inches trust me girls love bigger ones i've had escapenumber times as many chicks since i used these pills a year ago the package i used was the escapenumber month supply one and its worth every cent and more the website is http ctmay com ring me on the weekend and we will go out and drink again a...
4,1,system of the home it will have the capabilities to be linked far as i know and what i am doing within it as a part of it so with respect to the affects of technology on society we have science ad agencies are cashin g in on its' commerciality and photographs and paint on electronic canvases it still seems like silence white out black out lights out it didn't happen although far from p erfect especially in that it precludes a vast explanation for this i can' t understand how people can rely so avant gardes of the art world additionally writers and lawyers yet to re ach its full potential i...
...,...,...
247515,0,on escapenumber escapenumber escapenumber rob dixon wrote snip for my elem doc firstchild elem elem elem nextsibling snip i covered that in my earlier email weirder stuff that you only tend to see people coming from a c background do for my node head node node node next but in perl it is rarely necessary to do this sort of loop since most functions return a list that can be iterated over using for for my node head nodes in this case your module should include a children method for my elem doc children or a true iterator my iter doc child iter while my elem iter next if it doesn't then bug ...
247516,1,we have everything you need escapelong cialescapenumbers sescapenumberft tescapenumberbs vescapenumberagra sescapenumberft tescapenumberbs cialescapenumbers vescapenumberagra levescapenumbertra propecescapenumbera valescapenumberum xanescapenumberx ambescapenumberen zybescapenumbern atarescapenumberx atescapenumbervan carescapenumbersoma ultrescapenumberm escapelong lipescapenumbertor merescapenumberdia zocescapenumberr nescapenumberrvasc we respect your privacy we guarantee you a total anonymity of your escapenumberrder visit us escapelong inc online at http www electioo com
247517,0,hi quick question say i have a date variable in a data frame or matrix and i'd like to preserve the date format when using write table however when i export the data i get the generic number underlying the date not the date per se and a number such as escapenumber escapenumber etc are not meaningful in excel is there any way i can preserve the format of a date on writing into a text file tia and best tir r help stat math ethz ch mailing list https stat ethz ch mailman listinfo r help please do read the posting guide http www r project org posting guide html and provide commented minimal se...
247518,1,thank you for your loan request which we recieved on escape

In [11]:
merged['label'].unique()

array([1, 0])

In [12]:
merged['text'].unique().shape

(193849,)

In [13]:
merged = merged.drop_duplicates().reset_index(drop=True)

In [14]:
merged = merged.dropna(subset=['text']).reset_index(drop=True)
merged = merged.drop_duplicates(subset=['text']).reset_index(drop=True)

In [15]:
merged.to_csv("data/merged_spam_ham.csv", index=False)

### Splitting

In [16]:
train_val, test = train_test_split(
    merged,
    train_size=0.9,
    stratify=merged['label'],  
    shuffle=True,
    random_state=42
)
train, val = train_test_split(
    train_val,
    train_size=0.8,
    stratify=train_val['label'],  
    shuffle=True,
    random_state=42
)

train = train.reset_index(drop=True) #72%
val = val.reset_index(drop=True) #18%
test = test.reset_index(drop=True) #10%


In [17]:
directory = "filtered_data"
try:
    os.mkdir(directory)
    print(f"Directory '{directory}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory}' already exists.")

Directory 'filtered_data' created successfully.


In [18]:
train.to_csv("filtered_data/spam_ham_train.csv", index=False)
val.to_csv("filtered_data/spam_ham_val.csv", index=False)
test.to_csv("filtered_data/spam_ham_test.csv", index=False)

---

In [19]:
train = pd.read_csv("filtered_data/spam_ham_train.csv")
val = pd.read_csv("filtered_data/spam_ham_val.csv")
test = pd.read_csv("filtered_data/spam_ham_test.csv")

## Classifier

In [ ]:
model_name = "bert-base-uncased" #test cased/uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [21]:
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [23]:
reduced_train = train.sample(n=2000, random_state=42)
reduced_val = val.sample(n=2000, random_state=42)
reduced_test = test.sample(n=2000, random_state=42)

In [24]:
train_ds = Dataset.from_pandas(reduced_train)
val_ds = Dataset.from_pandas(reduced_val)
test_ds = Dataset.from_pandas(reduced_test)

In [25]:
dataset_dict = DatasetDict({"train": train_ds, "val": val_ds, "test": test_ds})

In [26]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_data = dataset_dict.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

In [30]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy_res = accuracy.compute(predictions=predictions, references=labels)
    return {"accuracy" : accuracy_res["accuracy"]}

In [41]:
lr = 1e-4
batch_sz = 32
epoch = 3
wd = 0.01

training_args = TrainingArguments(
    output_dir="bert-spam-ham-classifier-added-wd",
    per_device_train_batch_size=batch_sz,
    per_device_eval_batch_size=batch_sz,
    num_train_epochs=epoch,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

    learning_rate= lr,
    weight_decay= wd,
    report_to="none"
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_36/1459056268.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.281700,0.275353,0.886000
2,0.267600,0.257131,0.898000
3,0.261400,0.254108,0.897000


TrainOutput(global_step=189, training_loss=0.27021718908239295, metrics={'train_runtime': 196.1396, 'train_samples_per_second': 30.59, 'train_steps_per_second': 0.964, 'total_flos': 1578666332160000.0, 'train_loss': 0.27021718908239295, 'epoch': 3.0})

In [ ]:
!pip3 install tqdm==4.62.1

### Testing

In [44]:
test_results = trainer.predict(tokenized_data["test"])

In [45]:
test_results.metrics

{'test_loss': 0.25653406977653503,
 'test_accuracy': 0.8955,
 'test_runtime': 31.395,
 'test_samples_per_second': 63.704,
 'test_steps_per_second': 2.007}

### Testing learning rate

In [49]:
results = []

batch_sz = 32
epoch = 3
learning_rates = [1e-5, 2e-5, 3e-5, 5e-5, 1e-4]
for lr in learning_rates:
    print(f"Testing learning rate: {lr}")
    
    training_args = TrainingArguments(
        output_dir="bert-spam-ham-classifier-testing-learning-rate",
        per_device_train_batch_size=batch_sz,
        per_device_eval_batch_size=batch_sz,
        num_train_epochs=epoch,
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    
        learning_rate= lr,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    test_results = trainer.predict(tokenized_data["test"])

    results.append({"learning_rate": lr, 
                    "loss": test_results.metrics["test_loss"],
                   "accuracy": test_results.metrics["test_accuracy"]})



print("----------------------")
results = sorted(results, key=lambda x: x["accuracy"], reverse=True)
print(results)
print("----------------------")

Testing learning rate: 1e-05


/tmp/ipykernel_36/3555311632.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.235800,0.255498,0.897500
2,0.244700,0.256383,0.895500
3,0.250400,0.255051,0.897000


Testing learning rate: 2e-05


/tmp/ipykernel_36/3555311632.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.219600,0.256017,0.897500
2,0.227700,0.244093,0.901000
3,0.232300,0.240670,0.902000


Testing learning rate: 0.0001


/tmp/ipykernel_36/3555311632.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.210600,0.250301,0.897500
2,0.218700,0.233723,0.905500
3,0.222400,0.230673,0.905500


----------------------
[{'learning_rate': 0.0001, 'loss': 0.2279825359582901, 'accuracy': 0.907}, {'learning_rate': 5e-05, 'loss': 0.23968440294265747, 'accuracy': 0.901}, {'learning_rate': 3e-05, 'loss': 0.24686606228351593, 'accuracy': 0.8965}, {'learning_rate': 2e-05, 'loss': 0.2521768808364868, 'accuracy': 0.8935}, {'learning_rate': 1e-05, 'loss': 0.25644001364707947, 'accuracy': 0.892}]
----------------------
